In [8]:
import ray

# Start Ray. If you're connecting to an existing cluster, you would use
# ray.init(address=<cluster-address>) instead.
ray.shutdown()
ray.init()

2021-12-05 13:59:32,015	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265
2021-12-05 13:59:35,615	ERROR import_thread.py:89 -- ImportThread: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 870, in run


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:59325',
 'raylet_socket_name': 'tcp://127.0.0.1:56567',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\James\\AppData\\Local\\Temp\\ray\\session_2021-12-05_13-59-25_570195_2912',
 'metrics_export_port': 62544,
 'node_id': 'bdeb0a06ccc564d67016ca7f211418a46ebc33ebf51e8a1d5f794ab2'}

    self._target(*self._args, **self._kwargs)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\worker.py", line 439, in print_logs
    pubsub_client.subscribe(gcs_utils.LOG_FILE_CHANNEL)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\client.py", line 1377, in subscribe
    ret_val = self.execute_command('SUBSCRIBE', *new_channels.keys())
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\client.py", line 1253, in execute_command
    self.connection = self.connection_pool.get_connection(
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\connection.py", line 1179, in get_connection
    if connection.can_read():
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\connection.py", line 743, in can_read
    return self._parser.can_read(timeout)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\connection.py", line 410, in can_read
    return self.read_from_socket(timeout=timeout,
  File "D:\anaconda3\envs\yourenvname\lib\

In [9]:
ray.is_initialized() 

True

Actors

In [14]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value

    def get_counter(self):
        return self.value

counter_actor = Counter.remote()
ray.get(counter_actor.increment.remote())

1

In [15]:
@ray.remote
class Foo(object):

    # Any method of the actor can return multiple object refs.
    @ray.method(num_returns=2)
    def bar(self):
        return 1, 2

f = Foo.remote()
obj_ref1, obj_ref2 = f.bar.remote()
ray.get(obj_ref1)

1

In [16]:
@ray.remote(num_cpus=2, num_gpus=1)
class GPUActor(object):
    pass

@ray.remote(num_cpus=4)
class Counter(object):
    pass

a1 = Counter.options(num_cpus=1, resources={"Custom1": 1}).remote()
a2 = Counter.options(num_cpus=2, resources={"Custom2": 1}).remote()
a3 = Counter.options(num_cpus=3, resources={"Custom3": 1}).remote()

In [ ]:
ray.actor.exit_actor()

In [18]:
import time

@ray.remote
def f(counter):
    for _ in range(1000):
        time.sleep(0.1)
        counter.increment.remote()
        
counter = Counter.remote()
# Start some tasks that use the actor.
[f.remote(counter) for _ in range(3)]
# Print the counter value.
for _ in range(10):
    time.sleep(1)
    print(ray.get(counter.get_counter.remote()))

0
10
19
39
59
89


KeyboardInterrupt: 

In [21]:
import ray

@ray.remote
class Actor:
    pass

# driver_1.py
# Job 1 creates an actor, "orange" in the "colors" namespace.
ray.init(address="auto", namespace="colors",ignore_reinit_error=True)
Actor.options(name="orange", lifetime="detached")

# driver_2.py
# Job 2 is now connecting to a different namespace.
ray.init(address="auto", namespace="fruit",ignore_reinit_error=True)
# This fails because "orange" was defined in the "colors" namespace.
ray.get_actor("orange")

# driver_3.py
# Job 3 connects to the original "colors" namespace
ray.init(address="auto", namespace="colors", ignore_reinit_error=True)
# This returns the "orange" actor we created in the first job.
ray.get_actor("orange")

2021-12-05 14:09:00,084	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379
2021-12-05 14:09:00,087	INFO worker.py:852 -- Calling ray.init() again after it has already been called.
2021-12-05 14:09:00,824	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379
2021-12-05 14:09:00,825	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


ValueError: Failed to look up actor with name 'orange'. This could because 1. You are trying to look up a named actor you didn't create. 2. The named actor died. 3. You did not use a namespace matching the namespace of the actor.

In [22]:
counter = Counter.options(name="CounterActor", lifetime="detached").remote()
counter = ray.get_actor("CounterActor")
print(ray.get(counter.get_counter.remote()))

2021-12-05 14:09:11,697	WARNING worker.py:1245 -- It looks like you're creating a detached actor in an anonymous namespace. In order to access this actor in the future, you will need to explicitly connect to this namespace with ray.init(namespace="023ddc17-a32a-4ceb-b0f9-c1d1ae42ef3c", ...)


KeyboardInterrupt: 

In [26]:
from ray.util import ActorPool

@ray.remote
class Actor:
    def double(self, n):
        return n * 2

a1, a2 = Actor.remote(), Actor.remote()
pool = ActorPool([a1, a2])

# pool.map(..) returns a Python generator object ActorPool.map
gen = pool.map(lambda a, v: a.double.remote(v), [1, 2, 3, 4])
print([v for v in gen])

[2, 4, 6, 8]


Namespaces

In [27]:
ray.shutdown()
ray.init(namespace="hello")
# or using ray client
#ray.init("ray://<head_node_host>:10001", namespace="world")

2021-12-05 14:11:38,124	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265
2021-12-05 14:11:41,775	ERROR import_thread.py:89 -- ImportThread: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread ray_listen_error_messages:
Traceback (most recent call last):


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:61777',
 'raylet_socket_name': 'tcp://127.0.0.1:61624',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\James\\AppData\\Local\\Temp\\ray\\session_2021-12-05_14-11-31_497014_2912',
 'metrics_export_port': 56978,
 'node_id': 'ee8dfcc48a85aeeb276268f4761f5ad84800a82706255b5c102c2544'}

  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
    self.run()
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 870, in run
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\worker.py", line 439, in print_logs
    pubsub_client.subscribe(gcs_utils.LOG_FILE_CHANNEL)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\client.py", line 1377, in subscribe
    self._target(*self._args, **self._kwargs)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\worker.py", line 1211, in listen_error_messages_raylet
    ret_val = self.execute_command('SUBSCRIBE', *new_channels.keys())
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\client.py", line 1253, in execute_command
        worker.error_message_pubsub_client.psubscribe(error_pubsub_channel)
  File "D:\anaconda3\envs\yo

In [30]:
# `ray start --head` has been run to launch a local cluster.
import ray

@ray.remote
class Actor:
    pass

# Job 1 creates two actors, "orange" and "purple" in the "colors" namespace.
with ray.init(namespace="colors", ignore_reinit_error=True):
    Actor.options(name="orange", lifetime="detached")
    Actor.options(name="purple", lifetime="detached")

# Job 2 is now connecting to a different namespace.
with ray.init(namespace="fruits", ignore_reinit_error=True):
  # This fails because "orange" was defined in the "colors" namespace.
    ray.get_actor("orange")
  # This succceeds because the name "orange" is unused in this namespace.
    Actor.options(name="orange", lifetime="detached")
    Actor.options(name="watermelon", lifetime="detached")

# Job 3 connects to the original "colors" namespace
context = ray.init("ray://localhost:10001", namespace="colors", ignore_reinit_error=True)
# This fails because "watermelon" was in the fruits namespace.
ray.get_actor("watermelon")
# This returns the "orange" actor we created in the first job, not the second.
ray.get_actor("orange")
context.disconnect()
# We are manually managing the scope of the connection in this example.

2021-12-05 17:42:53,732	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


AttributeError: __enter__

In [34]:
import ray

@ray.remote
class Actor:
    pass

# Job 1 connects to an anonymous namespace by default
ray.init(ignore_reinit_error=True)
Actor.options(name="my_actor", lifetime="detached")
print(ray.get_runtime_context().namespace)
ray.shutdown()

# Job 2 connects to a _different_ anonymous namespace by default
ray.init(ignore_reinit_error=True)
# This succeeds because the second job is in its own namespace.
Actor.options(name="my_actor", lifetime="detached")
ray.shutdown()

2021-12-05 17:45:05,863	INFO worker.py:852 -- Calling ray.init() again after it has already been called.
2021-12-05 17:45:26,124	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


Dependencies

In [40]:
import ray
import requests

# This example runs on a local machine, but you can also do
# ray.init(address=..., runtime_env=...) to connect to a cluster.
ray.shutdown()
ray.init(runtime_env={"pip": ["requests"]})

@ray.remote
def reqs():
    return requests.get("https://www.ray.io/")

print(ray.get(reqs.remote())) # <Response [200]>

2021-12-05 17:49:00,073	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


NotImplementedError: The 'pip' field in runtime_env is not currently supported on Windows.

AsyncIO 

In [41]:
import ray
import asyncio

ray.shutdown()
ray.init()

@ray.remote
class AsyncActor:
    # multiple invocation of this method can be running in
    # the event loop at the same time
    async def run_concurrent(self):
        print("started")
        await asyncio.sleep(2) # concurrent workload here
        print("finished")

actor = AsyncActor.remote()

# regular ray.get
ray.get([actor.run_concurrent.remote() for _ in range(4)])

# async ray.get
await actor.run_concurrent.remote()

2021-12-05 17:49:50,196	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


 pid=14712) started
 pid=14712) started
 pid=14712) started
 pid=14712) started
 pid=14712) finished
 pid=14712) finished
 pid=14712) finished
 pid=14712) finished
 pid=14712) started
 pid=14712) finished


In [42]:
@ray.remote
def some_task():
    return 1

await some_task.remote()
await asyncio.wait([some_task.remote()])

({<Task finished name='Task-5' coro=<_wrap_awaitable() done, defined at D:\anaconda3\envs\yourenvname\lib\asyncio\tasks.py:688> result=1>},
 set())

In [45]:
refs = [some_task.remote() for _ in range(4)]
futs = [ref.future() for ref in refs]
for fut in concurrent.futures.as_completed(futs):
    assert fut.done()
    print(fut.result())

NameError: name 'concurrent' is not defined

In [46]:
import asyncio

@ray.remote
class AsyncActor:
    async def run_task(self):
        print("started")
        await asyncio.sleep(1) # Network, I/O task here
        print("ended")

actor = AsyncActor.remote()
# All 50 tasks should start at once. After 1 second they should all finish.
# they should finish at the same time
ray.get([actor.run_task.remote() for _ in range(50)])

 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) ended
 pid=5260) ended
 pid=5260) ended

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [47]:
import asyncio

@ray.remote
class AsyncActor:
    async def run_task(self):
        print("started")
        await asyncio.sleep(1) # Network, I/O task here
        print("ended")

actor = AsyncActor.options(max_concurrency=10).remote()

# Only 10 tasks will be running concurrently. Once 10 finish, the next 10 should run.
ray.get([actor.run_task.remote() for _ in range(50)])

 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended


In [48]:
@ray.remote
class ThreadedActor:
    def task_1(self): print("I'm running in a thread!")
    def task_2(self): print("I'm running in another thread!")

a = ThreadedActor.options(max_concurrency=2).remote()
ray.get([a.task_1.remote(), a.task_2.remote()])

[None, None]

 pid=14660) I'm running in a thread!
 pid=14660) I'm running in another thread!


 Concurrency Per-Method with Concurrency Groups

In [49]:
@ray.remote(concurrency_groups={"io": 2, "compute": 4})
class AsyncIOActor:
    def __init__(self):
        pass

    @ray.method(concurrency_group="io")
    async def f1(self):
        pass

    @ray.method(concurrency_group="io")
    async def f2(self):
        pass

    @ray.method(concurrency_group="compute")
    async def f3(self):
        pass

    @ray.method(concurrency_group="compute")
    async def f4(self):
        pass

    async def f5(self):
        pass

a = AsyncIOActor.remote()
a.f1.remote()  # executed in the "io" group.
a.f2.remote()  # executed in the "io" group.
a.f3.remote()  # executed in the "compute" group.
a.f4.remote()  # executed in the "compute" group.
a.f5.remote()  # executed in the default group.

ObjectRef(20d3f3cc944a6233e072e42aac59f1e3af9289fc0100000001000000)

In [51]:
@ray.remote(concurrency_groups={"io": 2})
class AsyncIOActor:
    async def f1(self):
        pass

actor = AsyncIOActor.options(max_concurrency=10).remote()

GPU

In [54]:
import os

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))
    
use_gpu.remote()

ObjectRef(f74e3e0219621f0bffffffffffffffffffffffff0100000001000000)

 pid=16760) ray.get_gpu_ids(): [0]
 pid=16760) CUDA_VISIBLE_DEVICES: 0


Serialization

In [64]:
import numpy as np
obj = [np.zeros(42)] * 99
l = ray.get(ray.put(obj))
l[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [72]:
import ray
import sqlite3

ray.init(ignore_reinit_error=True)

class DBConnection:
    def __init__(self, path):
        self.path = path
        self.conn = sqlite3.connect(path)

    # without '__reduce__', the instance is unserializable.
    def __reduce__(self):
        deserializer = DBConnection
        serialized_data = (self.path,)
        return deserializer, serialized_data

original = DBConnection("db")
print(original.conn)

copied = ray.get(ray.put(original))
print(copied.conn)

2021-12-05 18:06:06,706	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


Memory

In [78]:
@ray.remote
def f(arg):
    return arg

a = ray.put(None)
b = f.remote(None)


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [79]:
a = ray.put(np.zeros(1))
b = ray.get(a)
del a
b

array([0.])

Placement

In [85]:
# Import placement group APIs.
from ray.util.placement_group import (
    placement_group,
    placement_group_table,
    remove_placement_group
)

# Initialize Ray.
ray.init(num_gpus=2, resources={"extra_resource": 2}, ignore_reinit_error=True)

bundle1 = {"GPU": 2}
bundle2 = {"extra_resource": 2}

pg = placement_group([bundle1, bundle2], strategy="STRICT_PACK")

2021-12-05 18:10:22,434	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


In [83]:
import ray
from pprint import pprint

# Import placement group APIs.
from ray.util.placement_group import (
    placement_group,
    placement_group_table,
    remove_placement_group
)

ray.init(num_gpus=2, resources={"extra_resource": 2}, ignore_reinit_error=True)

2021-12-05 18:09:58,292	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


In [89]:
# Two "CPU"s are available.
ray.init(num_cpus=2, ignore_reinit_error = True)

# Create a placement group.
pg = placement_group([{"CPU": 2}])
ray.get(pg.ready())

# Now, 2 CPUs are not available anymore because they are pre-reserved by the placement group.
@ray.remote(num_cpus=2)
def f():
    return True

# Won't be scheduled because there are no 2 cpus.
f.remote()

# Will be scheduled because 2 cpus are reserved by the placement group.
f.options(placement_group=pg).remote()

2021-12-05 18:11:25,837	INFO worker.py:852 -- Calling ray.init() again after it has already been called.
2021-12-05 18:11:35,863	WARNING worker.py:1245 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.


KeyboardInterrupt: 

In [90]:
# This API is asynchronous.
remove_placement_group(pg)

# Wait until placement group is killed.
import time
time.sleep(1)
# Check the placement group has died.
pprint(placement_group_table(pg))

"""
{'bundles': {0: {'GPU': 2.0}, 1: {'extra_resource': 2.0}},
'name': 'unnamed_group',
'placement_group_id': '40816b6ad474a6942b0edb45809b39c3',
'state': 'REMOVED',
'strategy': 'STRICT_PACK'}
"""

ray.shutdown()

{'bundles': {0: {'CPU': 2.0}},
 'name': '',
 'placement_group_id': '82c7fb6b47eca35dc0746796f80c22d4',
 'state': 'REMOVED',
 'stats': {'end_to_end_creation_latency_ms': 8.758,
           'highest_retry_delay_ms': 0.0,
           'scheduling_attempt': 1,
           'scheduling_latency_ms': 8.486,
           'scheduling_state': 'REMOVED'},
 'strategy': 'PACK'}


Debugging

In [94]:
import cProfile  # Added import statement

def ex1():
    list1 = []
    for i in range(5):
        list1.append(ray.get(some_task.remote()))

def main():
    ray.init(ignore_reinit_error=True)
    cProfile.run('ex1()')  # Modified call to ex1

if __name__ == "__main__":
    main()

2021-12-05 18:20:04,613	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


         734 function calls (714 primitive calls) in 0.032 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.031    0.031 1435182158.py:3(ex1)
        1    0.000    0.000    0.031    0.031 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
       10    0.000    0.000    0.000    0.000 _collections_abc.py:657(get)
    20/10    0.000    0.000    0.000    0.000 client_mode_hook.py:110(client_mode_should_convert)
    15/10    0.000    0.000    0.030    0.003 client_mode_hook.py:98(wrapper)
        5    0.000    0.000    0.000    0.000 conda.py:206(get_uri)
    15/10    0.000    0.000    0.000    0.000 copy.py:128(deepcopy)
        5    0.000    0.000    0.000    0.000 copy.py:182(_deepcopy_atomic)
        5    0.000    0.000    0.000    0.000 copy.py:226(_deepcopy_dict)
       10    0.000    0.000    0.000    0.000 copy.py:242(_keep_alive)
        5    0.000 

In [98]:
# Our actor
@ray.remote
class Sleeper(object):
    def __init__(self):
        self.sleepValue = 0.5

    # Equivalent to func(), but defined within an actor
    def actor_func(self):
        time.sleep(self.sleepValue)
        
def ex4():
    # This is suboptimal in Ray, and should only be used for the sake of this example
    actor_example = Sleeper.remote()

    five_results = []
    for i in range(5):
        five_results.append(actor_example.actor_func.remote())

    # Wait until the end to call ray.get()
    ray.get(five_results)
    
def main():
    ray.init(ignore_reinit_error=True)
    cProfile.run('ex4()')

if __name__ == "__main__":
    main()

2021-12-05 18:20:50,867	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


         9460 function calls (9450 primitive calls) in 2.543 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.543    2.543 1627753386.py:11(ex4)
        1    0.000    0.000    2.543    2.543 <string>:1(<module>)
      269    0.000    0.000    0.000    0.000 <string>:1(__new__)
        1    0.000    0.000    0.000    0.000 __init__.py:1276(disable)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1675(getEffectiveLevel)
        1    0.000    0.000    0.000    0.000 __init__.py:1689(isEnabledFor)
        2    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
        1    0.000    0.000    0.000    0.000 __init__.py:218(_acquireLock)
        1    0.000    0.000    0.000    0.000 __init__.py:227(_releaseLock)
       13    0.000    0.000    0.000    0.000 __init__.py:889(__missing__)
       27    0.000    0.000   

Fault tolerance

In [100]:
import numpy as np
import os
import ray
import time

ray.init(ignore_reinit_error=True)

@ray.remote(max_retries=1)
def potentially_fail(failure_probability):
    time.sleep(0.2)
    if np.random.random() < failure_probability:
        os._exit(0)
    return 0

for _ in range(3):
    try:
        # If this task crashes, Ray will retry it up to one additional
        # time. If either of the attempts succeeds, the call to ray.get
        # below will return normally. Otherwise, it will raise an
        # exception.
        ray.get(potentially_fail.remote(0.5))
        print('SUCCESS')
    except ray.exceptions.WorkerCrashedError:
        print('FAILURE')

2021-12-05 18:21:54,002	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


SUCCESS
SUCCESS
SUCCESS


In [101]:
import os
import ray
import time

ray.init(ignore_reinit_error=True)

@ray.remote(max_restarts=5)
class Actor:
    def __init__(self):
        self.counter = 0

    def increment_and_possibly_fail(self):
        self.counter += 1
        time.sleep(0.2)
        if self.counter == 10:
            os._exit(0)
        return self.counter

actor = Actor.remote()

# The actor will be restarted up to 5 times. After that, methods will
# always raise a `RayActorError` exception. The actor is restarted by
# rerunning its constructor. Methods that were sent or executing when the
# actor died will also raise a `RayActorError` exception.
for _ in range(100):
    try:
        counter = ray.get(actor.increment_and_possibly_fail.remote())
        print(counter)
    except ray.exceptions.RayActorError:
        print('FAILURE')

2021-12-05 18:22:17,451	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


1
2
3
4
5
6
7
8
9
FAILURE
1
2
3
4
5
6
7
8
9
FAILURE
1
2
3
4
5
6
7
8
9
FAILURE
1
2
3
4
5
6
7
8
9
FAILURE
1
2
3
4
5
6
7
8
9
FAILURE
1
2
3
4
5
6
7
8
9
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE
FAILURE


In [103]:
import os
import ray

ray.init(ignore_reinit_error=True)

@ray.remote(max_restarts=5, max_task_retries=-1)
class Actor:
    def __init__(self):
        self.counter = 0

    def increment_and_possibly_fail(self):
        # Exit after every 10 tasks.
        if self.counter == 10:
            os._exit(0)
        self.counter += 1
        return self.counter

actor = Actor.remote()

# The actor will be reconstructed up to 5 times. The actor is
# reconstructed by rerunning its constructor. Methods that were
# executing when the actor died will be retried and will not
# raise a `RayActorError`. Retried methods may execute twice, once
# on the failed actor and a second time on the restarted actor.
for _ in range(50):
    counter = ray.get(actor.increment_and_possibly_fail.remote())
    print(counter)  # Prints the sequence 1-10 5 times.

# After the actor has been restarted 5 times, all subsequent methods will
# raise a `RayActorError`.
for _ in range(10):
    try:
        counter = ray.get(actor.increment_and_possibly_fail.remote())
        print(counter)  # Unreachable.
    except ray.exceptions.RayActorError:
        print('FAILURE')  # Prints 10 times.

2021-12-05 18:22:47,801	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10


Job Submission

In [104]:
from ray.dashboard.modules.job.sdk import JobSubmissionClient

client = JobSubmissionClient("http://127.0.0.1:8265")

In [106]:
from ray.dashboard.modules.job.common import JobStatus, JobStatusInfo

job_id = 1
def wait_until_finish(job_id):
    start = time.time()
    timeout = 5
    while time.time() - start <= timeout:
        status_info = client.get_job_status(job_id)
        status = status_info.status
        print(f"status: {status}")
        if status in {JobStatus.SUCCEEDED, JobStatus.STOPPED, JobStatus.FAILED}:
            break
        time.sleep(1)


wait_until_finish(job_id)
logs = client.get_job_logs(job_id)

RuntimeError: Request failed with status code 404: Job 1 does not exist.

Patterns

In [108]:
@ray.remote(num_cpus=1)
class Worker:
    def work(self):
        return "done"

@ray.remote(num_cpus=1)
class Supervisor:
    def __init__(self):
        self.workers = [Worker.remote() for _ in range(3)]
    def work(self):
        return ray.get([w.work.remote() for w in self.workers])

ray.init(ignore_reinit_error=True)
sup = Supervisor.remote()
print(ray.get(sup.work.remote()))  # outputs ['done', 'done', 'done']

2021-12-05 18:29:39,046	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


['done', 'done', 'done']


In [109]:
@ray.remote
def map(obj, f):
    return f(obj)

items = list(range(100))
map_func = lambda i : i*2
output = ray.get([map.remote(i, map_func) for i in items])

@ray.remote
def map(obj, f):
    return f(obj)
@ray.remote
def sum_results(*elements):
    return np.sum(elements)

items = list(range(100))
map_func = lambda i : i*2
remote_elements = [map.remote(i, map_func) for i in items]

# simple reduce
remote_final_sum = sum_results.remote(*remote_elements)
result = ray.get(remote_final_sum)

# tree reduce
intermediate_results = [sum_results.remote(
    *remote_elements[i * 20: (i + 1) * 20]) for i in range(5)]
remote_final_sum = sum_results.remote(*intermediate_results)
result = ray.get(remote_final_sum)

KeyboardInterrupt: 

In [110]:
import ray

@ray.remote
class GlobalVarActor:
    def __init__(self):
        self.global_v = 3
    def set_global_v(self, v):
        self.global_v = v
    def get_global_v(self):
        return self.global_v

@ray.remote
class A:
    def __init__(self, global_v_registry):
        self.global_v_registry = global_v_registry
    def f(self):
        return ray.get(self.global_v_registry.get_global_v.remote()) + 3

global_v_registry = GlobalVarActor.remote()
actor = A.remote(global_v_registry)
ray.get(global_v_registry.set_global_v.remote(4))
# This will print 7 correctly.
print(ray.get(actor.f.remote()))

7
